In [2]:
import yfinance as yf
import pandas as pd
import numpy as np



In [6]:
tickers = "AAPL NVDA MSFT GOOG META AMZN"
data = yf.download(tickers, period="max", interval="1h", group_by='ticker', threads=True, auto_adjust=True)



[*********************100%***********************]  6 of 6 completed


In [13]:
data['AAPL'].head()

Price,Open,High,Low,Close,Volume
Datetime,,,,,
2022-10-24 13:30:00+00:00,147.250000,148.350006,146.000000,146.630203,20169319
2022-10-24 14:30:00+00:00,146.630096,148.449997,146.470001,148.210007,10131448
2022-10-24 15:30:00+00:00,148.219894,148.820007,147.529999,148.436096,7921370
2022-10-24 16:30:00+00:00,148.434998,148.729996,147.910004,148.080002,6399968
2022-10-24 17:30:00+00:00,148.070007,149.365005,147.929993,149.285004,5909741


In [7]:
data.shape

(3491, 30)

In [21]:
data.columns

MultiIndex([('AAPL',   'Open'),
            ('AAPL',   'High'),
            ('AAPL',    'Low'),
            ('AAPL',  'Close'),
            ('AAPL', 'Volume'),
            ('MSFT',   'Open'),
            ('MSFT',   'High'),
            ('MSFT',    'Low'),
            ('MSFT',  'Close'),
            ('MSFT', 'Volume'),
            ('NVDA',   'Open'),
            ('NVDA',   'High'),
            ('NVDA',    'Low'),
            ('NVDA',  'Close'),
            ('NVDA', 'Volume')],
           names=['Ticker', 'Price'])

In [8]:
# Save the data
for symbol in tickers.split(" "):
    symbol_df = data[symbol]
    symbol_df.to_csv(f'{symbol}_hourly_data.csv')
    print(f"Data for {symbol} saved to '{symbol}_hourly_data.csv'")

Data for AAPL saved to 'AAPL_hourly_data.csv'
Data for NVDA saved to 'NVDA_hourly_data.csv'
Data for MSFT saved to 'MSFT_hourly_data.csv'
Data for GOOG saved to 'GOOG_hourly_data.csv'
Data for META saved to 'META_hourly_data.csv'
Data for AMZN saved to 'AMZN_hourly_data.csv'


In [24]:
print(data.index.names)

['Datetime']


In [25]:
print(data.columns)

MultiIndex([('AAPL',   'Open'),
            ('AAPL',   'High'),
            ('AAPL',    'Low'),
            ('AAPL',  'Close'),
            ('AAPL', 'Volume'),
            ('MSFT',   'Open'),
            ('MSFT',   'High'),
            ('MSFT',    'Low'),
            ('MSFT',  'Close'),
            ('MSFT', 'Volume'),
            ('NVDA',   'Open'),
            ('NVDA',   'High'),
            ('NVDA',    'Low'),
            ('NVDA',  'Close'),
            ('NVDA', 'Volume')],
           names=['Ticker', 'Price'])


In [9]:
data_long = data.stack(level='Ticker').reset_index()
data_long.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
data_long.head()

/tmp/ipykernel_3595/4140012834.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data_long = data.stack(level='Ticker').reset_index()


,Date,Ticker,Open,High,Low,Close,Volume
0,2022-10-24 13:30:00+00:00,AAPL,147.250000,148.350006,146.000000,146.630203,20169319
1,2022-10-24 13:30:00+00:00,AMZN,119.250000,120.389999,116.570000,117.127502,14337761
2,2022-10-24 13:30:00+00:00,GOOG,101.500000,102.349899,100.300003,100.839996,6057487
3,2022-10-24 13:30:00+00:00,META,127.250000,127.375000,124.570000,126.389999,12673848
4,2022-10-24 13:30:00+00:00,MSFT,243.648895,245.729996,241.294998,242.479996,6780395


In [10]:
data_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20946 entries, 0 to 20945
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    20946 non-null  datetime64[ns, UTC]
 1   Ticker  20946 non-null  object             
 2   Open    20946 non-null  float64            
 3   High    20946 non-null  float64            
 4   Low     20946 non-null  float64            
 5   Close   20946 non-null  float64            
 6   Volume  20946 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 1.1+ MB


In [11]:
def engineer_features(df):
    df = df.sort_values(by=['Ticker', 'Date'])

    # Returns
    df['Returns'] = df.groupby('Ticker')['Close'].pct_change()
    df['Log_Returns'] = df.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))

    # Volume features
    df['Volume_Change'] = df.groupby('Ticker')['Volume'].pct_change()
    df['Volume_Change'] = df['Volume_Change'].clip(lower=-5, upper=5)
    df['Volume_MA_Ratio'] = df.groupby('Ticker')['Volume'].transform(lambda x: x / x.rolling(window=10).mean())

    # Price momentum
    for window in [1, 3, 5, 10]:
        df[f'Momentum_{window}'] = df.groupby('Ticker')['Close'].transform(lambda x: x.pct_change(periods=window))

    # Close Position to High and Low
    df['Bar_Range'] = df['High'] - df['Low']
    df['Close_Position'] = (df['Close'] - df['Low']) / df['Bar_Range']
    # if high and low are equal, set close position to 0.5
    df.loc[df['Bar_Range'] == 0, 'Close_Position'] = 0.5
    df['Close_Position_Category'] = pd.cut(df['Close_Position'], 
                                           bins=[0, 0.2, 0.4, 0.6, 0.8, 1], 
                                           labels=['Very Low', 'Low', 'Middle', 'High', 'Very High'])

    # Our label: 1 if next hour's price is higher, else 0
    df['Target'] = df.groupby('Ticker')['Close'].transform(lambda x: (x.shift(-1) > x).astype(int))

    # Drop NaN values
    df.dropna(inplace=True)


    return df

df = engineer_features(data_long)



In [34]:
df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Returns,Log_Returns,Volume_Change,Volume_MA_Ratio,Momentum_1,Momentum_3,Momentum_5,Momentum_10,Bar_Range,Close_Position,Close_Position_Category,Target
30,2022-10-25 16:30:00+00:00,AAPL,151.789993,151.839996,151.149994,151.494995,5889941,-0.002009,-0.002011,-0.278226,0.647903,-0.002009,0.002283,0.010101,0.033177,0.690002,0.500000,Middle,1
33,2022-10-25 17:30:00+00:00,AAPL,151.490005,152.050003,151.154999,152.049301,6496406,0.003659,0.003652,0.102966,0.744380,0.003659,0.003063,0.016644,0.025904,0.895004,0.999216,Very High,1
36,2022-10-25 18:30:00+00:00,AAPL,152.044998,152.300003,151.759995,152.268707,6861939,0.001443,0.001442,0.056267,0.795926,0.001443,0.003088,0.007401,0.025820,0.540009,0.942046,Very High,1
39,2022-10-25 19:30:00+00:00,AAPL,152.264999,152.490005,152.020004,152.380005,7608674,0.000731,0.000731,0.108823,0.870339,0.000731,0.005842,0.005245,0.029038,0.470001,0.765957,High,0
42,2022-10-26 13:30:00+00:00,AAPL,150.960007,151.320007,149.529999,150.660004,22487941,-0.011288,-0.011352,1.955566,2.162298,-0.011288,-0.009137,-0.007510,0.009211,1.790009,0.631285,High,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20779 entries, 60 to 20945
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   Date                     20779 non-null  datetime64[ns, UTC]
 1   Ticker                   20779 non-null  object             
 2   Open                     20779 non-null  float64            
 3   High                     20779 non-null  float64            
 4   Low                      20779 non-null  float64            
 5   Close                    20779 non-null  float64            
 6   Volume                   20779 non-null  int64              
 7   Returns                  20779 non-null  float64            
 8   Log_Returns              20779 non-null  float64            
 9   Volume_Change            20779 non-null  float64            
 10  Volume_MA_Ratio          20779 non-null  float64            
 11  Momentum_1               20779 n

In [13]:
df.to_csv('hourly_data_features.csv', index=False)

In [3]:
df = pd.read_csv('data/hourly_data_features.csv')

In [4]:
df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Returns,Log_Returns,Volume_Change,Volume_MA_Ratio,Momentum_1,Momentum_3,Momentum_5,Momentum_10,Bar_Range,Close_Position,Close_Position_Category,Target
0,2022-10-25 16:30:00+00:00,AAPL,151.789993,151.839996,151.149994,151.494995,5889941,-0.002009,-0.002011,-0.278226,0.647903,-0.002009,0.002283,0.010101,0.033177,0.690002,0.500000,Middle,1
1,2022-10-25 17:30:00+00:00,AAPL,151.490005,152.050003,151.154999,152.049301,6496406,0.003659,0.003652,0.102966,0.744380,0.003659,0.003063,0.016644,0.025904,0.895004,0.999216,Very High,1
2,2022-10-25 18:30:00+00:00,AAPL,152.044998,152.300003,151.759995,152.268707,6861939,0.001443,0.001442,0.056267,0.795926,0.001443,0.003088,0.007401,0.025820,0.540009,0.942046,Very High,1
3,2022-10-25 19:30:00+00:00,AAPL,152.264999,152.490005,152.020004,152.380005,7608674,0.000731,0.000731,0.108823,0.870339,0.000731,0.005842,0.005245,0.029038,0.470001,0.765957,High,0
4,2022-10-26 13:30:00+00:00,AAPL,150.960007,151.320007,149.529999,150.660004,22487941,-0.011288,-0.011352,1.955566,2.162298,-0.011288,-0.009137,-0.007510,0.009211,1.790009,0.631285,High,1


In [5]:
df.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Returns',
       'Log_Returns', 'Volume_Change', 'Volume_MA_Ratio', 'Momentum_1',
       'Momentum_3', 'Momentum_5', 'Momentum_10', 'Bar_Range',
       'Close_Position', 'Close_Position_Category', 'Target'],
      dtype='object')